# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [89]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [90]:
Vacation_pd = pd.read_csv("Resources/Weather_Output.csv")
Vacation_pd


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Guerrero Negro,27.9769,-114.0611,65.77,65,55,11.43,MX,1605236400
1,1,Vaini,-21.2000,-175.2000,79.74,84,58,13.98,TO,1605236400
2,2,Birao,10.2849,22.7882,71.87,24,5,6.62,CF,1605236400
3,3,New Norfolk,-42.7826,147.0587,70.74,47,76,7.61,AU,1605236400
4,4,Inongo,-1.9500,18.2667,70.75,94,97,4.27,CD,1605236400
...,...,...,...,...,...,...,...,...,...,...
555,555,Hof,50.3167,11.9167,40.69,90,69,5.06,DE,1605236400
556,556,Kruisfontein,-34.0033,24.7314,61.18,88,0,5.93,ZA,1605236400
557,557,Kalat,29.0225,66.5916,51.71,47,15,4.88,PK,1605236400
558,558,Vestmanna,62.1564,-7.1664,48.61,91,100,19.69,FO,1605236400


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [91]:
gmaps.configure(api_key=g_key)
locations = Vacation_pd[["Lat", "Lng"]]
Humidity = Vacation_pd["Humidity"].astype(float)

In [92]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [93]:
Vacation_conditional_df = Vacation_pd.loc[(Vacation_pd["Max Temp"] >=75) & (Vacation_pd["Max Temp"] <=80) ]
#Vacation_pd.isna
Vacation_clean_df=Vacation_conditional_df.dropna()
Vacation_clean_df
#Vacation_conditional_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Vaini,-21.2000,-175.2000,79.74,84,58,13.98,TO,1605236400
12,12,Pyu,18.4833,96.4333,79.00,81,17,5.79,MM,1605236400
15,15,Kapaa,22.0752,-159.3190,79.25,68,29,17.02,US,1605236400
31,31,Touros,-5.1989,-35.4608,78.76,77,10,14.54,BR,1605236400
34,34,Faanui,-16.4833,-151.7500,79.90,73,72,9.93,PF,1605236400
...,...,...,...,...,...,...,...,...,...,...
538,538,Takoradze,4.8845,-1.7554,78.71,86,1,4.54,GH,1605236400
543,543,Bonthe,7.5264,-12.5050,78.82,86,0,1.57,SL,1605236400
545,545,Nicoya,10.1483,-85.4520,76.59,94,87,3.02,CR,1605236400
551,551,Kiama,-34.6833,150.8667,77.61,62,99,8.46,AU,1605236400


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [94]:
hotel_df = Vacation_clean_df
hotel_df["Hotel Name"] = ""
hotel_df 



,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,Vaini,-21.2000,-175.2000,79.74,84,58,13.98,TO,1605236400,
12,12,Pyu,18.4833,96.4333,79.00,81,17,5.79,MM,1605236400,
15,15,Kapaa,22.0752,-159.3190,79.25,68,29,17.02,US,1605236400,
31,31,Touros,-5.1989,-35.4608,78.76,77,10,14.54,BR,1605236400,
34,34,Faanui,-16.4833,-151.7500,79.90,73,72,9.93,PF,1605236400,
...,...,...,...,...,...,...,...,...,...,...,...
538,538,Takoradze,4.8845,-1.7554,78.71,86,1,4.54,GH,1605236400,
543,543,Bonthe,7.5264,-12.5050,78.82,86,0,1.57,SL,1605236400,
545,545,Nicoya,10.1483,-85.4520,76.59,94,87,3.02,CR,1605236400,
551,551,Kiama,-34.6833,150.8667,77.61,62,99,8.46,AU,1605236400,


In [95]:
Hotel_location = Vacation_clean_df[["Lat", "Lng"]]
Hotel_location

,Lat,Lng
1,-21.2000,-175.2000
12,18.4833,96.4333
15,22.0752,-159.3190
31,-5.1989,-35.4608
34,-16.4833,-151.7500
...,...,...
538,4.8845,-1.7554
543,7.5264,-12.5050
545,10.1483,-85.4520
551,-34.6833,150.8667


In [96]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}" 
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #location=location&radius=5000&type=lodging&key=g_key"

    name_address = requests.get(base_url, params=params)

    name_address = name_address.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
         print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [98]:
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,1,Vaini,-21.2000,-175.2000,79.74,84,58,13.98,TO,1605236400,Keleti Beach Resort
12,12,Pyu,18.4833,96.4333,79.00,81,17,5.79,MM,1605236400,Mr Bo Bo
15,15,Kapaa,22.0752,-159.3190,79.25,68,29,17.02,US,1605236400,Sheraton Kauai Resort at Coconut Beach
31,31,Touros,-5.1989,-35.4608,78.76,77,10,14.54,BR,1605236400,INN NEW HORIZON
34,34,Faanui,-16.4833,-151.7500,79.90,73,72,9.93,PF,1605236400,Bora-Bora Pearl Beach Resort & Spa


In [99]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [101]:
# Add marker layer ontop of heat map
#info_box_template = """
#<dl>
#<dt>Name</dt><dd>{Hotel Name}</dd>
#<dt>City</dt><dd>{City}</dd>
#<dt>Country</dt><dd>{Country}</dd>
#</dl>
#"""
#hotel_layer = gmaps.symbol_layer(
    #locations, fill_color='rgba(0, 150, 0, 0.4)',
    #stroke_color='rgba(0, 0, 150, 0.4)', scale=2, info_box_content=[f"Hotel info: {hotel_df}" for index, row in hotel_df.iterrows()])
#Hotel_info = markers = gmaps.marker_layer(locations)
Hotel_marker = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(Hotel_marker)
#fig.add_layer(heat_layer)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))